In [9]:
!pip install thop
!pip install torchsummary

In [10]:
import torch
import torch.nn as nn
from thop import profile
from torchsummary import summary
import torch.nn.functional as F

class CustomAlexNet(nn.Module):
    def __init__(self):
        super(CustomAlexNet, self).__init__()
        # Create your AlexNet
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, padding=2, stride=4)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(in_features=256*6*6, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=1024)
        self.fc3 = nn.Linear(in_features=1024, out_features=1000)

    def forward(self, x):
        # Connect your Model
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomAlexNet().to(device)
inputs = torch.randn(1, 3, 224, 224).to(device)
flops, params = profile(model, inputs=(inputs,))

print(f"FLOPS: {flops}")
print(f"Parameters: {params}")


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPS: 698533568.0
Parameters: 45442856.0


In [11]:
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
            Conv2d-2          [-1, 192, 27, 27]         307,392
            Conv2d-3          [-1, 384, 13, 13]         663,936
            Conv2d-4          [-1, 256, 13, 13]         884,992
            Conv2d-5          [-1, 256, 13, 13]         590,080
            Linear-6                 [-1, 4096]      37,752,832
            Linear-7                 [-1, 1024]       4,195,328
            Linear-8                 [-1, 1000]       1,025,000
Total params: 45,442,856
Trainable params: 45,442,856
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 3.75
Params size (MB): 173.35
Estimated Total Size (MB): 177.67
----------------------------------------------------------------
